## Imports

In [2]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
import pyautogui as pag
import time
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
import json
import pandas as pd

## Request

In [3]:
os.environ['PATH'] += r"D:/Anaconda/anaconda3"
driver = webdriver.Firefox()
driver.maximize_window()

In [4]:
driver.get('https://permits.cityofboise.org/CitizenAccess/Welcome.aspx')



In [5]:
link = driver.find_element(By.XPATH, '/html/body/form/div[3]/header/div/nav/div/div/ul/li[2]/span/div')
link.click()

WebDriverWait(driver,30).until(
    EC.text_to_be_present_in_element(
        (By.XPATH,'/html/body/form/div[3]/header/div/nav/div/div/ul/li[2]/div/a[5]'),
        'Licenses'
    )
)

lk2 = driver.find_element(By.XPATH, '//*[@id="cobnav"]/div/ul/li[2]/div/a[2]') 

lk2.click()



In [6]:
rec_type = driver.find_element(By.ID, 'ctl00_PlaceHolderMain_generalSearchForm_ddlGSPermitType')



rec_type.click()

sel_rec = driver.find_element(By.XPATH, '/html/body/form/div[4]/div/div[7]/div[1]/table/tbody/tr/td/div[2]/div[3]/div/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[1]/div[2]/table/tbody/tr/td/div/div[2]/table/tbody/tr/td[1]/select/option[2]')

sel_rec.click()


## Calendar

In [7]:
scroll_origin = ScrollOrigin.from_viewport(10, 10)
ActionChains(driver)\
.scroll_from_origin(scroll_origin, 0, 800)\
.perform()

In [8]:
time.sleep(4)

In [9]:

pag.moveTo(480,250)
pag.click()
time.sleep(1)
pag.moveTo(440,280)
pag.click()
time.sleep(1)
pag.moveTo(390,350)
pag.click()
time.sleep(1)
pag.moveTo(430,340)
pag.click()
time.sleep(1)

In [10]:
find_eng = driver.find_element(By.ID, 'ctl00_PlaceHolderMain_btnNewSearch')
driver.execute_script("document.getElementById('ctl00_PlaceHolderMain_btnNewSearch').click()")

## Search

In [11]:
scroll_origin = ScrollOrigin.from_viewport(10, 10)
ActionChains(driver)\
.scroll_from_origin(scroll_origin, 0, -10000)\
.perform()

## Permits scraping

In [12]:
all_permits = {'0':{}}

In [20]:
## Status

def p_status (value):
    global status
    status = None

    try:
        status = driver.find_element(By.ID, f"ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList_ctl{value}_lblStatus").text
        
        
    except:
        pass

## Project Name

def p_name (value):
    global project_name
    project_name = None

    try:
        project_name = driver.find_element(By.ID, f"ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList_ctl{value}_lblProjectName").text

    except:
        pass

## Permit Number
def p_num (value):
    global permit_num
    permit_num = None

    if value is not None:
        try:
            permit_num = driver.find_element(By.ID, f"ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList_ctl{value}_lblPermitNumber1").text
            print(permit_num, '| Features: High')
            return True
            
        except:
           permit_num = driver.find_element(By.ID, f"ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList_ctl{value}_lblPermitNumber").text
           print(permit_num, '| Features: Low')
           return False
        

def go_to_link(value):

    if p_num(value) == True:
        try:
            driver.execute_script(f"document.getElementById('ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList_ctl{value}_hlPermitNumber1').click()")
            return True
        except:
            driver.execute_script(f"document.getElementById('ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList_ctl{value}_hlPermitNumber').click()")
            return True
    else:
        return False



## Project attributes
     
def attributes(value):


    global project_st
    global project_city
    global licenced_professional
    global professional_city
    global professional_zip
    global professional_street
    global applicant_name
    global applicant_company
    global applicant_main_phone
    global applicant_email   
    
    
    project_st = None
    project_city = None
    licenced_professional = None
    professional_city = None
    professional_zip = None
    professional_street =None
    applicant_name = None
    applicant_company = None
    applicant_main_phone = None
    applicant_email = None


    if go_to_link(value) == True:
        time.sleep(7)
        try:
            project_st = driver.find_element(By.CLASS_NAME, f"contactinfo_addressline1").text

        except:
            project_st = driver.find_element(By.CLASS_NAME, f"contactinfo_addressline2").text


        
        
        ## Project City
        try:
            project_city = driver.find_element(By.CLASS_NAME, f"contactinfo_region").text

        except:
            pass

        

        ## Applicant Name
        try:
            apl_fname = driver.find_element(By.CLASS_NAME, f"contactinfo_firstname").text
            apl_lname = driver.find_element(By.CLASS_NAME, f"contactinfo_lastname").text

            applicant_name = apl_fname + ' ' + apl_lname
            

        except:
            pass
            
        
        ## Applicant Company Name
        try:
            applicant_company = driver.find_element(By.CLASS_NAME, f"contactinfo_businessname").text

        except:
            pass


        ## Applicant Phone
        try:
            applicant_main_phone = driver.find_element(By.CLASS_NAME, f"contactinfo_phone1").text

        except:
            pass


        ## Applicant Email
        try:
            applicant_email = driver.find_element(By.CLASS_NAME, f"contactinfo_email").text

        except:
            pass


        ## Licensed Professional
        try:
            table = driver.find_elements(By.CLASS_NAME, f"table_child")[2].text
        except:
            pass
        
        
        splitted = table.split(sep="\n")

        if len(splitted) >= 3:
            try:
                if ':' in splitted[3].split(sep=',')[0].strip():
                    professional_city=(splitted[2].split(sep=',')[0])
                    professional_street=(splitted[1])
                    licenced_professional=(splitted[0])    
                else:
                    professional_city=(splitted[3].split(sep=',')[0])
                    professional_street=(splitted[2])
                    licenced_professional=(splitted[1])                

            except:
                pass

                
            try:
                splitted[3].split(sep=',')[2]
                professional_zip=(splitted[3].split(sep=',')[2])
            except:
                professional_zip=(splitted[2].split(sep=',')[2])

            else:
                pass
            
            driver.back()

        else:
            
            driver.back()


def permits():
    # for i in range(len(values)):
        global permits_data

        permits_data = {}
        permits_data['permit_num']= permit_num
        permits_data['project_name']= project_name
        permits_data['status']= status
        permits_data['project_st']= project_st
        permits_data['project_city']= project_city
        permits_data['licenced_professional']= licenced_professional
        permits_data['professional_city']= professional_city
        permits_data['professional_zip']= professional_zip
        permits_data['professional_street']= professional_street
        permits_data['applicant_name']= applicant_name
        permits_data['applicant_company']= applicant_company
        permits_data['applicant_main_phone']= applicant_main_phone
        permits_data['applicant_email']= applicant_email
        add_permits(permits_data)


def add_permits(permits_data):
    all_permits.update({f'{i}': (permits_data)})


In [15]:
values = ['02', '04','05','06']#,'03','06','07', '08','09', '10','11', '12','13']
for i, value in enumerate(values):
    print('Working on {}...'.format(value))
    p_status(value)
    p_name(value)
    attributes(value)
    permits()
    add_permits(permits_data)


Working on 02...
24TMP06227 Low info
Working on 04...
BLD24-00418 High info
Working on 05...
BLD24-00415 High info
Working on 06...
24TMP06058 Low info


## Output

In [16]:
print(all_permits)


{'0': {'permit_num': '24TMP06227', 'project_name': '4388 W Hill Rd', 'status': None, 'project_st': None, 'project_city': None, 'licenced_professional': None, 'professional_city': None, 'professional_zip': None, 'professional_street': None, 'applicant_name': None, 'applicant_company': None, 'applicant_main_phone': None, 'applicant_email': None}, '1': {'permit_num': 'BLD24-00418', 'project_name': 'Harris Ranch North #4 LOT 1 BLOCK 4', 'status': 'Applicant Upload', 'project_st': '729 S. Bridgeway Pl.', 'project_city': 'Eagle,', 'licenced_professional': None, 'professional_city': None, 'professional_zip': None, 'professional_street': None, 'applicant_name': 'Kyle Davis', 'applicant_company': 'Boise Hunter Homes', 'applicant_main_phone': 'Main Phone:\n208-577-5501', 'applicant_email': 'bhhdrafting@boisehunterhomes.com'}, '2': {'permit_num': 'BLD24-00415', 'project_name': 'Wolke Residence', 'status': 'Applicant Upload', 'project_st': '1456 E IRON EAGLE DR', 'project_city': 'EAGLE,', 'licence

In [17]:
r = json.dumps(all_permits)
load = json.loads(r)


In [18]:
df = pd.json_normalize(load['0'])


In [19]:
for i in range (0,len(load.keys())-1):
    a = (pd.json_normalize(load[f'{i+1}']))
    b = [df, a]
    output = pd.concat (b)
    df = output

display(output)



## Export
# output.to_csv

,permit_num,project_name,status,project_st,project_city,licenced_professional,professional_city,professional_zip,professional_street,applicant_name,applicant_company,applicant_main_phone,applicant_email
0,24TMP06227,4388 W Hill Rd,None,None,None,None,None,None,None,None,None,None,None
0,BLD24-00418,Harris Ranch North #4 LOT 1 BLOCK 4,Applicant Upload,729 S. Bridgeway Pl.,"Eagle,",None,None,None,None,Kyle Davis,Boise Hunter Homes,Main Phone:\n208-577-5501,bhhdrafting@boisehunterhomes.com
0,BLD24-00415,Wolke Residence,Applicant Upload,1456 E IRON EAGLE DR,"EAGLE,",SHERBURNE-MARRS LLC,EAGLE,83616,1456 E IRON EAGLE DR,Desiree Hammond,SHERBURNE-MARRS LLC,Main Phone:\n208-939-9221,desiree@sherburne-marrs.com
0,24TMP06058,,None,None,None,None,None,None,None,None,None,None,None
